SETTING UP THE ENVIRONMENT

In [2]:
!wget https://www.dropbox.com/s/fxn3ldztzwxm0rw/FruitsData.zip

--2023-09-24 06:36:21--  https://www.dropbox.com/s/fxn3ldztzwxm0rw/FruitsData.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/fxn3ldztzwxm0rw/FruitsData.zip [following]
--2023-09-24 06:36:21--  https://www.dropbox.com/s/raw/fxn3ldztzwxm0rw/FruitsData.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucac5dc2cc56af18c838e798abfd.dl.dropboxusercontent.com/cd/0/inline/CEV0hBtX4StrD0qCxJFJLQiYd5h0EC0nGlj-Wlv8ka63Uz3qbiTSiXe2Wn7dK9CU-LFpVb3BfmWLdrr4i0hKkzEM2hNyj-PYw-NsMdaLcMZK06HhdXkREfqNwaJNjquifcX-XsY8P2HrHS_qg50Q2ww6/file# [following]
--2023-09-24 06:36:21--  https://ucac5dc2cc56af18c838e798abfd.dl.dropboxusercontent.com/cd/0/inline/CEV0hBtX4StrD0qCxJFJLQiYd5h0EC0nGlj-Wlv8ka63Uz3qbiTSiXe2Wn7dK9CU-LFpVb3BfmWLdrr4i0hKkzEM2hNyj-PY

In [3]:
!unzip -q "/content/FruitsData.zip"

In [4]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

tf.random.set_seed(4)

CREATING THE PATH OBJECTS

In [23]:
train_path = Path("fruits-360/Training")
test_path = Path("fruits-360/Test")

In [6]:
train_image_paths = list(train_path.glob("*/*"))
train_image_paths = list(map(lambda x: str(x), train_image_paths))


FETCHING LABELS

In [7]:
def get_labels(image_path):
    return image_path.split("/")[-2]

train_image_labels = list(map(lambda x : get_labels(x), train_image_paths))

ENCODING THE LABELS

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

train_image_labels = le.fit_transform(train_image_labels)


CREATING ONE-HOT ENCODING FOR FETCHED LABELS

In [9]:
train_image_labels = tf.keras.utils.to_categorical(train_image_labels)


TRAIN VALIDATION SPLIT

In [10]:
from sklearn.model_selection import train_test_split

Train_paths, Val_paths, Train_labels, Val_labels = train_test_split(train_image_paths,
                                                                   train_image_labels)



CREATING A TENSOR OBJECT FOR IMAGES

In [11]:
def load(image, label):
    image = tf.io.read_file(image)
    image = tf.io.decode_jpeg(image, channels = 3)
    return image, label

DATA AUGMENTATION AND RESIZING

In [12]:
IMG_SIZE = 224
BATCH_SIZE = 32

resize = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Resizing(IMG_SIZE, IMG_SIZE)
])

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    tf.keras.layers.experimental.preprocessing.RandomZoom(height_factor = (-0.3, -0.2))


])

CREATING TENSORFLOW DATASET

In [13]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
def get_dataset(paths, labels, train = True):
    image_paths = tf.convert_to_tensor(paths)
    labels = tf.convert_to_tensor(labels)

    image_dataset = tf.data.Dataset.from_tensor_slices(image_paths)
    label_dataset = tf.data.Dataset.from_tensor_slices(labels)

    dataset = tf.data.Dataset.zip((image_dataset, label_dataset))

    dataset = dataset.map(lambda image, label: load(image, label))
    dataset = dataset.map(lambda image, label: (resize(image), label),
                          num_parallel_calls = AUTOTUNE)
    dataset = dataset.shuffle(1000)
    dataset = dataset.batch(BATCH_SIZE)

    if train:
        dataset = dataset.map(lambda image, label: (data_augmentation(image), label),
                          num_parallel_calls = AUTOTUNE)
    dataset = dataset.repeat()
    return dataset

In [14]:
train_dataset = get_dataset(Train_paths, Train_labels)
val_dataset = get_dataset(Val_paths,  Val_labels, train = False)

MODEL BUILDING

In [15]:
from tensorflow.keras.applications import ResNet50V2
backbone = ResNet50V2(
          input_shape = (224,224,3),
          include_top = False
)

model = tf.keras.Sequential([
    backbone,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(131, activation = "softmax")
])

model.summary()

94668760/94668760 [==============================] - 0s 0us/step
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 7, 7, 2048)        23564800  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 131)               268419    
                                                                 
Total params: 23833219 (90.92 MB)
Trainable params: 23787779 (90.74 MB)
Non-trainable params: 45440 (177.50 KB)
_________________________________________________________________


In [16]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001, beta_1 = 0.9, beta_2 = 0.999),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy', tf.keras.metrics.Precision(name = 'precision'),
               tf.keras.metrics.Recall(name = 'recall')]

   )

TRAINING THE MODEL

In [17]:
history1 = model.fit(
          train_dataset,
          steps_per_epoch=len(Train_paths)//BATCH_SIZE,
          epochs = 1,
          validation_data = val_dataset,
          validation_steps=len(Val_paths)//BATCH_SIZE,
          verbose = 1
         )

1586/1586 [==============================] - 668s 394ms/step - loss: 0.4455 - accuracy: 0.8755 - precision: 0.9344 - recall: 0.8409 - val_loss: 3.9551 - val_accuracy: 0.4772 - val_precision: 0.5165 - val_recall: 0.4533


In [18]:
model.layers[0].trainable = False

ADDING CALLBACKS

In [19]:
checkpoint = tf.keras.callbacks.ModelCheckpoint("best_weights.h5", verbose = 1, save_best_only = True,
                                               save_weights_only = True)
early_stop = tf.keras.callbacks.EarlyStopping(patience=4)


In [20]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 7, 7, 2048)        23564800  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 131)               268419    
                                                                 
Total params: 23833219 (90.92 MB)
Trainable params: 268419 (1.02 MB)
Non-trainable params: 23564800 (89.89 MB)
_________________________________________________________________


In [21]:
history2 = model.fit(train_dataset,
          steps_per_epoch=len(Train_paths)//BATCH_SIZE,
          epochs = 6,
          callbacks=[checkpoint, early_stop],
          validation_data = val_dataset,
          validation_steps=len(Val_paths)//BATCH_SIZE,
          verbose = 1
         )

Epoch 1/6
1586/1586 [==============================] - ETA: 0s - loss: 0.0906 - accuracy: 0.9715 - precision: 0.9750 - recall: 0.9683
Epoch 1: val_loss improved from inf to 7.98989, saving model to best_weights.h5
1586/1586 [==============================] - 634s 399ms/step - loss: 0.0906 - accuracy: 0.9715 - precision: 0.9750 - recall: 0.9683 - val_loss: 7.9899 - val_accuracy: 0.2411 - val_precision: 0.2571 - val_recall: 0.2310
Epoch 2/6
1586/1586 [==============================] - ETA: 0s - loss: 0.0566 - accuracy: 0.9833 - precision: 0.9849 - recall: 0.9815
Epoch 2: val_loss improved from 7.98989 to 2.64036, saving model to best_weights.h5
1586/1586 [==============================] - 617s 387ms/step - loss: 0.0566 - accuracy: 0.9833 - precision: 0.9849 - recall: 0.9815 - val_loss: 2.6404 - val_accuracy: 0.5515 - val_precision: 0.5799 - val_recall: 0.5328
Epoch 3/6
1586/1586 [==============================] - ETA: 0s - loss: 0.0480 - accuracy: 0.9847 - precision: 0.9860 - recall: 0.9

TESTING

In [24]:
test_image_paths = list(test_path.glob('*/*'))
test_image_paths = list(map(lambda x: str(x), test_image_paths))
test_labels = list(map(lambda x : get_labels(x), test_image_paths))

test_labels = le.fit_transform(test_labels)
test_labels = tf.keras.utils.to_categorical(test_labels)


test_image_paths = tf.convert_to_tensor(test_image_paths)
test_labels = tf.convert_to_tensor(test_labels)

def decode_image(image, label):
    image = tf.io.read_file(image)
    image = tf.io.decode_jpeg(image, channels = 3)
    image = tf.image.resize(image, [224,224], method = "bilinear")
    return image, label

test_dataset = (
      tf.data.Dataset
     .from_tensor_slices((test_image_paths, test_labels))
     .map(decode_image)
     .batch(BATCH_SIZE)
)

In [27]:

loss, acc, prec, rec = model.evaluate(test_dataset)

print("loss is: ", loss)
print("acc is: ", acc)
print("prec is: ", prec)
print("rec is: ", rec)

709/709 [==============================] - 65s 92ms/step - loss: 6.4739 - accuracy: 0.3724 - precision: 0.3886 - recall: 0.3627
loss is:  6.473881244659424
acc is:  0.3724435865879059
prec is:  0.3885819613933563
rec is:  0.36270275712013245
